# HW4 - Pandas exercises

We open with some imports as usual - some of these will be unfamiliar, but bear with me

In [1]:
import urllib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris


# Some helpful functions 
just use these as be and don't worry too much about about them for now, we will learn how they work soon

In [2]:
# Same data manipulation to get USGS streamflow as
# a pandas dataframe as before
def create_usgs_url(site_no, begin_date, end_date):
    return (
        f'https://waterdata.usgs.gov/nwis/dv?'
        f'cb_00060=on&format=rdb&referred_module=sw&'
        f'site_no={site_no}&'
        f'begin_date={begin_date}&'
        f'end_date={end_date}'
    )

def open_usgs_data(site, begin_date, end_date):
    url = create_usgs_url((site), begin_date, end_date)
    response = urllib.request.urlopen(url)
    df = pd.read_table(
        response,
        comment='#',
        skipfooter=1,
        delim_whitespace=True,
        names=['agency', 'site', 'date', 'streamflow', 'quality_flag'],
        index_col=2,
        parse_dates=True
    ).iloc[2:]

    # Now convert the streamflow data to floats and
    # the index to datetimes. When processing raw data
    # it's common to have to do some extra postprocessing
    df['streamflow'] = df['streamflow'].astype(np.float64)
    df.index = pd.DatetimeIndex(df.index)
    return df

def open_daymet_data(lat, lon, begin_date, end_date):
    args = {'lat':  lat, 'lon': lon, 'format': 'csv',
            'start': begin_date, 'end': end_date}
    query = urllib.parse.urlencode(args)
    url = f"https://daymet.ornl.gov/single-pixel/api/data?{query}"
    response = urllib.request.urlopen(url)
    df = pd.read_csv(response, header=6)
    datestring = (df['year'].astype(str) + df['yday'].astype(str))
    dates = pd.to_datetime(datestring, format='%Y%j')
    df.index = pd.DatetimeIndex(dates)
    return df


# Additionally, setting some "metadata" - this will become second nature soon

In [3]:

site = '09506000'
begin_date = '1992-09-25'
end_date = '2022-09-25'
lat = 34.4483605
lon = -111.7898705


# Now open up some datasets from the above commands, you are almost to your exercises

In [4]:

verde_df = open_daymet_data(lat, lon, begin_date, end_date)
usgs_df = open_usgs_data(site, begin_date, end_date)
verde_df = verde_df.reindex(verde_df.index)
verde_df['streamflow'] = usgs_df['streamflow']
verde_df.head()


/tmp/ipykernel_1047/63972079.py:15: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  df = pd.read_table(


,year,yday,dayl (s),prcp (mm/day),srad (W/m^2),swe (kg/m^2),tmax (deg c),tmin (deg c),vp (Pa),streamflow
1992-09-25,1992,269,42469.79,0.0,413.81,0.0,33.57,12.92,1489.09,129.0
1992-09-26,1992,270,42337.25,0.0,419.69,0.0,34.03,12.61,1459.42,121.0
1992-09-27,1992,271,42204.86,0.0,436.54,0.0,34.89,11.41,1348.48,122.0
1992-09-28,1992,272,42072.64,0.0,421.55,0.0,35.44,12.82,1479.16,127.0
1992-09-29,1992,273,41940.63,0.0,380.67,0.0,33.42,14.18,1616.07,120.0


# 1. How do you see a quick summary of what is in `verde_df`?

In [5]:
# TODO: your code here

# 2. How do you get a listing of the columns in `verde_df`?

In [ ]:
# TODO: your code here

# 3. How do you select the streamflow column in `verde_df`?

In [ ]:
# TODO: your code here

# 4. How do you plot the streamflow in `verde_df`?

In [ ]:
# TODO: your code here

# 5. How do you get the last streamflow value from `verde_df`?

In [ ]:
# TODO: your code here

# 6. What is the mean streamflow value for the 30 year period?

In [ ]:
# TODO: your code here

# 7. What is the maximum value for the 30 year period?


In [ ]:
# TODO: your code here

# 8. How do you find the maximum streamflow value for each year?

In [ ]:
# TODO: your code here

# 9. How do you make a scatter plot of `dayl (s)` versus `tmax (deg c)`?
#### INFO: `dayl` is the day length in seconds and `tmax` is the daily maximum temperature

In [ ]:
# TODO: your code here

# 10. How do you calculate (and plot) the mean daily minimum temperature for each day of year?  And plot it?
#### INFO: Daily minimum temperature is in the column `tmin (deg c)`

In [ ]:
# TODO: your code here

# 11. What is the average value of all columns for October 10 across all years?
#### INFO: October 10 is the 283rd day of year

In [ ]:
# TODO: your code here

# NEW DATASET : Iris
Loading data - here I provide a dataset for you
to work with for the first set of exercises

The iris dataset is a classic and very easy 
multi-class classification dataset. It describes
measurments of sepal & petal width/length for three
different species of iris

In [2]:
d = load_iris()
iris_df = pd.DataFrame(d['data'], columns=d['feature_names'])
iris_df.index = pd.Series(
    pd.Categorical.from_codes(d.target, d.target_names),
    name='species'
)
iris_df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
species,,,,
setosa,5.1,3.5,1.4,0.2
setosa,4.9,3.0,1.4,0.2
setosa,4.7,3.2,1.3,0.2
setosa,4.6,3.1,1.5,0.2
setosa,5.0,3.6,1.4,0.2


In [4]:
pd.unique(iris_df)

ValueError: could not broadcast input array from shape (150,4) into shape (150,)

# 12. How do you view the "unique" species in the `iris_df` index?


In [ ]:
# TODO: your code here

# 13. How do you "locate" only rows for the `versicolor` species?

In [ ]:
# TODO: your code here

# 14. How do you group by the 3 different species and take the mean across the whole dataframe?

In [ ]:
# TODO: your code here

# 15. How do you make a scatter plot of the `sepal length (cm)` versus the `petal length (cm)` for the `versicolor`` species?

In [ ]:
# TODO: your code here

# BONUS OPTION: 
 Do the same plot for `setosa` and `virginica` all on the same figure. Color them 'tomato', 'darkcyan', and 'darkviolet', respectively.  Worth 1 point extra credit


In [ ]:
# TODO: your code here